In [14]:
import gc
gc.collect()

668

In [8]:
import os
import cv2
import numpy as np
import csv
import shutil

# Extract Frames

In [9]:
DATA_DIR = 'Data'
DATA2_DIR = 'Data2'
CATEGORIES = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
NUM_CLASSES = len(CATEGORIES)
SEQUENCE_LENGTH = 80


In [10]:
def extract_frames(video_file, category):
    cap = cv2.VideoCapture(video_file)
    frames = []
    vid_name = os.path.splitext(os.path.basename(video_file))[0]
    frame_dir = os.path.join(DATA2_DIR, category, os.path.splitext(os.path.basename(video_file))[0])
    loaded = True
    
    with open(os.path.join(DATA2_DIR, 'Skip_vid_report.csv'), mode='a', newline='') as file1:
        with open(os.path.join(DATA2_DIR, 'Dataset_frames_report.csv'), mode='a', newline='') as file2:
            writer1 = csv.writer(file1)
            writer2 = csv.writer(file2)
            video_frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            if video_frames_count < SEQUENCE_LENGTH:
                writer1.writerow([category, video_file, str(video_frames_count)])
                loaded = False
                return 0, loaded
                
            else:
                os.makedirs(frame_dir, exist_ok=True)
                writer2.writerow([category, video_file, str(video_frames_count)])

                skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

                with open(os.path.join(DATA2_DIR, 'frames.csv'), mode='a', newline='') as file3:
                    writer3 = csv.writer(file3)

                    for frame_counter in range(SEQUENCE_LENGTH):

                        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

                        success, frame = cap.read()

                        if not success:
                            print("Fail to read")
                            break

                        cv2.imwrite(os.path.join(frame_dir, f'{frame_counter}.jpg'), frame)
                        writer3.writerow([category, vid_name, os.path.join(frame_dir, f'{frame_counter}.jpg')])

                        frames.append(frame)

                    cap.release()
            
    return frames, loaded

In [11]:
def preprocess_data():
    
    with open(os.path.join(DATA2_DIR, 'Extract_frame_report.csv'), mode='a', newline='') as file:
        writer3 = csv.writer(file)
        writer3.writerow(['Category', 'load_count', 'fail_count', 'load_error'])

        for i, category in enumerate(CATEGORIES):

            load_count = 0
            fail_count = 0
            load_error = 0
            category_dir = os.path.join(DATA_DIR, category)

            print('Loading ', category, ' : \n')

            for video_file in os.listdir(category_dir):
                video_path = os.path.join(category_dir, video_file)
                frames, loaded = extract_frames(video_path, category)
                if not loaded:
                    fail_count += 1
                    print('Fail to load ', video_file)
                else:
                    if len(frames) == SEQUENCE_LENGTH:
                        print('Succeed to load ', video_file)
                        load_count += 1
                    else:
                        load_error += 1
                        print(video_file, ' Loaded with error')
            writer3.writerow([category, load_count, fail_count, load_error])
            
    return 'Finish loading!'

In [12]:
preprocess_data()

Loading  dining  : 

Succeed to load  cDwhA4sOB_U_000233_000243.mp4
Succeed to load  cHbZpVSgQc4_000277_000287.mp4
Succeed to load  Ck8QyMOzx1E_000082_000092.mp4
Succeed to load  CmfZh6Dys-U_000123_000133.mp4
Succeed to load  coEYm2t6a8k_000760_000770.mp4
Succeed to load  cpdDHOanLbw_000001_000011.mp4
Succeed to load  d1jRCjy7azk_000010_000020.mp4
Succeed to load  d32vKHQ8XIk_000105_000115.mp4
Succeed to load  d6_PIEGXmx0_000202_000212.mp4
Succeed to load  DAclVsqLV1I_000079_000089.mp4
Succeed to load  DBK-2GKqYdM_000006_000016.mp4
Succeed to load  DBOGAsPQOSU_000577_000587.mp4
Succeed to load  DdDY8_QI7FM_000040_000050.mp4
Succeed to load  DEsVfpui5pU_000060_000070.mp4
Succeed to load  DhOjVjnLAdw_000025_000035.mp4
Succeed to load  dLh1eFp1WeA_000010_000020.mp4
Succeed to load  dngUFhkwDp8_000004_000014.mp4
Succeed to load  DoeZdW-IzWk_000052_000062.mp4
Succeed to load  dpzQNoqf6S0_000189_000199.mp4
Succeed to load  E5YOOZgZz0o_000000_000010.mp4
Succeed to load  ea5DQ9ziN-s_000220_000

Succeed to load  vxv2C6r_QQs_000142_000152.mp4
Succeed to load  vZ89J383p4U_000249_000259.mp4
Succeed to load  wcLKNfMMjKw_000213_000223.mp4
Succeed to load  wlosQx_ZD10_000074_000084.mp4
Succeed to load  woZ5QG8L22s_000122_000132.mp4
Succeed to load  wVYjvHw5pBY_000183_000193.mp4
Succeed to load  wz4BXSTT6c4_000020_000030.mp4
Succeed to load  X96TLlOX_VM_000010_000020.mp4
Succeed to load  xpk1rm771aU_000039_000049.mp4
Succeed to load  y1olrHlArxc_000687_000697.mp4
Succeed to load  Y6ujfOvvykc_000012_000022.mp4
Succeed to load  yBIsDAgMX0U_000174_000184.mp4
Succeed to load  yGeRwHCnuf4_000066_000076.mp4
Succeed to load  Yp5P1b2mK1Y_000115_000125.mp4
Succeed to load  Yu7ld_ded5I_000260_000270.mp4
Succeed to load  zrIVKRK_13Y_000079_000089.mp4
Succeed to load  zvKoKYxNPwY_000002_000012.mp4
Succeed to load  zwETBqDlmLo_000004_000014.mp4
Succeed to load  _JbHIkKPBhY_000191_000201.mp4
Succeed to load  _mX8LGvqp5E_000004_000014.mp4
Succeed to load  _t2jCSL0RU4_000000_000010.mp4
Loading  runn

'Finish loading!'

In [15]:
gc.collect()

0